In [12]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

In [13]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [14]:
import os
from pyngrok import ngrok

In [18]:
ngrok.kill()

In [20]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://tracy-nondoubtable-rose.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://tracy-nondoubtable-rose.ngrok-free.dev


True

In [21]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [22]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [23]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於臺灣新竹縣新豐鄉的私立科技大學。學校的創辦宗旨是配合國家經濟發展、培育工業專業人才。

**發展沿革：**
明新科大於1966年創校，當時名為「明新工業專科學校」，設有機械、土木、工業管理等科別。此後逐步增設電機、化工、資訊管理等科系，並於1993年更名為「明新工商專科學校」。1997年，學校改制為「明新技術學院」，並陸續增設旅館事業管理、幼兒保育、休閒事業管理、老人服務事業管理等系，奠定其服務事業學院的基礎。2002年9月，學校奉教育部核准升格為「明新科技大學」。2018年12月，更名為「明新學校財團法人明新科技大學」。

**校名與願景：**
「明新」二字取自《大學》「在明明德，在新民，在止於至善」的精義，旨在闡揚人類與生俱來的德性與情操，期許學子涵養高尚品德、擁有專業學問與優良技術，並在天人物我的平衡關係中，臻至全人之境。學校的校訓為「堅毅、求新、創造」。其發展願景是成為「一流產業大學」，教育目標則是培養「具實務經驗與人文素養之專業人才」以及「跨域整合、務實創新、全人學習」的專業人才。

**學術單位與特色：**
目前，明新科技大學設有六個學院，包括半導體學院、工程學院、管理學院、民生學院、人文與設計學院以及共同教育學院。涵括20個學系、2個學位學程（含1個博士學位學程）及11個碩士班。學校積極掌握世界與產業趨勢，致力於創新優化學校方針，並以「一流產業大學」為發展定位。

明新科大以其「學系含金量高、產業最愛特色大學」而聞名，在《遠見》雜誌的「企業最愛公私立技職科大調查」中曾獲得兩項企業最愛，且起薪排名私校第一，優於國立科大。

學校的特色包括：
*   **深耕產業合作：** 鄰近桃竹苗產業中心，與國際封測大廠合作成立「半導體封裝測試產業聯盟」，為半導體產業聚落提供優質人才。
*   **前瞻領域發展：** 鎖定半導體、AI、元宇宙、風電綠能等前瞻產業，發展多元學習、全球視野、永續經營與技術創新等四大育才特色。
*   **數位科技整合：** 成立「AI教育推動中心」與「AI產學推動中心」，將程式設計與人工智慧列為大一必修學分，並建置AI應用與創客教室。同時，也將生成式AI與AI專案應用導入「永續智慧商務」教學與實習場域，發展智

In [25]:
result2 = stateful_query("校長是誰？")
print(result2)

None


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
